In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%%capture
%run "4. Limpiar y estructurar datos.ipynb"

In [ ]:
def Codificar_Variables_Categoricas_A_Numericas(Variables_X):

    """
    Transforma variables categóricas en representaciones numéricas utilizando
    conversión directa y codificación por etiquetas (Label Encoding) como
    estrategia de respaldo.
    
    Esta función procesa sistemáticamente todas las columnas de tipo objeto
    en el DataFrame, intentando primero una conversión directa a valores
    numéricos. Para variables que no pueden convertirse directamente,
    aplica Label Encoding que asigna valores enteros únicos a cada
    categoría, preservando toda la información categórica en formato
    numérico compatible con algoritmos de machine learning.
    
    Parámetros:
    -----------
    Variables_X : pandas.DataFrame
        DataFrame que contiene las variables independientes, incluyendo
        columnas de tipo objeto (categóricas) que necesitan ser convertidas
        a formato numérico.
    
    Retorna:
    --------
    pandas.DataFrame
        DataFrame con todas las variables convertidas a tipos numéricos.
        Las variables originalmente numéricas permanecen inalteradas,
        mientras que las categóricas son transformadas usando la estrategia
        más apropiada (conversión directa o Label Encoding).
    
    Ejemplo de uso:
    ---------------
    >>> import pandas as pd
    >>> Datos = pd.DataFrame({
    ...     'Edad': [25, 30, 35],
    ...     'Ciudad': ['Madrid', 'Barcelona', 'Madrid'],
    ...     'Numeros_Texto': ['1', '2', '3']
    ... })
    >>> Datos_Numericos = Codificar_Variables_Categoricas_A_Numericas(Datos)
    >>> print(Datos_Numericos.dtypes)  # Todas numéricas
    
    Notas:
    ------
    - La función imprime el progreso de conversión para cada variable categórica.
    - Estrategia en dos pasos:
      1. Conversión directa con pd.to_numeric() para strings numéricos
      2. Label Encoding para categóricas verdaderas
    - Los valores faltantes en variables categóricas se marcan temporalmente
      con -999 antes del Label Encoding para evitar errores.
    - El Label Encoding asigna números enteros consecutivos (0, 1, 2, ...)
      a cada categoría única.
    - No se aplica One-Hot Encoding para evitar la explosión dimensional.
    
    """
    
    Variables_X_Convertidas = Variables_X.copy()
    
    for Columna in Variables_X.columns:
        if Variables_X[Columna].dtype == 'object':
            print(f"Convirtiendo {Columna} a numérica...")
            
            # Intentar convertir a numérica directamente.
            Variables_X_Convertidas[Columna] = pd.to_numeric(
                Variables_X[Columna], errors='coerce')
            
            # Si quedan valores no numéricos, usar LabelEncoder.
            if Variables_X_Convertidas[Columna].isnull().any():
                from sklearn.preprocessing import LabelEncoder
                Encoder = LabelEncoder()
                Valores_Validos = Variables_X[Columna].dropna()
                if len(Valores_Validos) > 0:
                    Variables_X_Convertidas[Columna] = Variables_X_Convertidas[Columna].fillna(-999)
                    Variables_X_Convertidas[Columna] = Encoder.fit_transform(
                        Variables_X_Convertidas[Columna].astype(str))
    
    return Variables_X_Convertidas

In [ ]:
def Codificar_Variables_Booleanas_A_Numericas(Variables_X):

    """
    Transforma variables booleanas (True/False) a representación entera
    binaria (1/0) para garantizar compatibilidad con librerías de modelado
    estadístico que requieren entrada numérica.
    
    Esta función identifica automáticamente todas las columnas de tipo
    booleano en el DataFrame y las convierte a enteros binarios manteniendo
    la semántica original: True se convierte en 1 y False en 0. Esta
    transformación es esencial para la compatibilidad con statsmodels y
    otros paquetes de análisis estadístico que no procesan directamente
    tipos booleanos de pandas.
    
    Parámetros:
    -----------
    Variables_X : pandas.DataFrame
        DataFrame que contiene las variables independientes, incluyendo
        posibles columnas de tipo booleano que necesitan transformación
        a formato entero binario.
    
    Retorna:
    --------
    pandas.DataFrame
        DataFrame idéntico al original pero con todas las variables
        booleanas convertidas a enteros (0 y 1). Las demás columnas
        permanecen completamente inalteradas en tipo y contenido.
    
    Ejemplo de uso:
    ---------------
    >>> import pandas as pd
    >>> Datos = pd.DataFrame({
    ...     'edad': [25, 30, 35],
    ...     'es_cliente': [True, False, True],
    ...     'tiene_descuento': [False, True, False]
    ... })
    >>> Datos_Numericos = Codificar_Variables_Booleanas_A_Numericas(Datos)
    >>> print(Datos_Numericos['Es_Cliente'].dtype)  # int64
    >>> print(Datos_Numericos['Es_Cliente'].tolist())  # [1, 0, 1]
    
    Notas:
    ------
    - La función imprime el número total de variables booleanas procesadas.
    - Conversión estándar: True → 1, False → 0
    - No afecta variables de otros tipos de datos (numéricas, categóricas, etc.).
    - Esencial para compatibilidad con statsmodels.OLS y modelos similares.
    - La transformación preserva completamente la información original.
    - Maneja automáticamente valores faltantes si existen en columnas booleanas.
    
    """
    
    Variables_X_Convertidas = Variables_X.copy()
    
    Variables_Booleanas = []
    for Columna in Variables_X.columns:
        if Variables_X[Columna].dtype == 'bool':
            Variables_Booleanas.append(Columna)
            Variables_X_Convertidas[Columna] = Variables_X[Columna].astype(int)
    
    print(f"Variables booleanas convertidas a enteros: {len(Variables_Booleanas)}")
    
    return Variables_X_Convertidas